In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
from mlxtend.data import loadlocal_mnist
import tensorflow as tf

In [8]:
def sigmoid(x,derivative=False):
    temp = 1/(1+np.exp(-x))
    if(derivative==True):
        return temp*(1-temp)
    return temp

def softmax(x, derivative=False):
    if derivative:
        s = softmax(x)
        return np.diag(s) - np.outer(s, s)
    else:
        exp_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
        return exp_x / np.sum(exp_x, axis=-1, keepdims=True)

def loss(predicted_output, desired_output):
    return 1/2*(desired_output-predicted_output)**2

class NeuralNetwork():
  def __init__(self,inputLayer,hiddenLayer,OutputLayer):
      self.inputLayer = inputLayer
      self.hiddenLayer = hiddenLayer
      self.ouputLayer = OutputLayer
      self.learningRate = 0.1

      self.hidden_weights = np.random.randn(hiddenLayer,inputLayer)*np.sqrt(2/inputLayer)
      self.hidden_bias = np.zeros([hiddenLayer,1])
      self.output_weights = np.random.randn(OutputLayer,hiddenLayer)
      self.output_bias = np.zeros([OutputLayer,1])
      self.loss = []

  def train(self,inputs,desired_output):
    hidden_layer_in = np.dot(self.hidden_weights,inputs) + self.hidden_bias
    hidden_layer_out = sigmoid(hidden_layer_in)

    #pass on to next layer
    output_layer_in = np.dot(self.output_weights,hidden_layer_out)+self.output_bias
    predicted_output = sigmoid(output_layer_in)


    #back-propagation
    error = desired_output - predicted_output
    d_predicted_output = error*sigmoid(predicted_output,derivative=True)

    error_hidden_layer = d_predicted_output.T.dot(self.output_weights)
    d_hidden_layer = error_hidden_layer.T*sigmoid(hidden_layer_out,derivative=True)

    self.output_weights += hidden_layer_out.dot(d_predicted_output.T).T*self.learningRate
    self.output_bias += np.sum(d_predicted_output,axis=0,keepdims=True)

    self.hidden_weights += inputs.dot(d_hidden_layer.T).T*self.learningRate
    self.hidden_bias += np.sum(d_hidden_layer,axis=0,keepdims=True)

    self.loss.append(loss(predicted_output,desired_output))

  def predict(self,inputs):
    hidden_layer_in = np.dot(self.hidden_weights,inputs) + self.hidden_bias
    hidden_layer_out = sigmoid(hidden_layer_in)

    #pass on to next layer
    output_layer_in = np.dot(self.output_weights,hidden_layer_out)+self.output_bias
    return sigmoid(output_layer_in)

In [9]:
X, y = loadlocal_mnist(images_path='/content/drive/MyDrive/train-images.idx3-ubyte',labels_path='/content/drive/MyDrive/train-labels.idx1-ubyte')
#Spliting dataset
num_train = 50000
num_test = 10000
X_train = X[:num_train, :]/255
y_train = np.zeros((num_train, 10))
y_train[np.arange(0, num_train) ,y[:num_train]] = 1
X_test = X[num_train:, :]/255
y_test = np.zeros((num_test, 10))
y_test[np.arange(0, num_test) ,y[y.size - num_test:]] = 1
print(X_train.shape,X_test.shape,sep = '\n')

(50000, 784)
(10000, 784)


In [14]:
nn = NeuralNetwork(784,280,10)

#Train model ------------------[HEHE]
for i in range(X_train.shape[0]):
  inputs = np.array(X_train[i, :].reshape(-1,1))
  desired_output = np.array(y_train[i, :].reshape(-1,1))
  nn.train(inputs,desired_output)

In [15]:
#Test it ------------------------- {How much accurate it is?(LOL)}
prediction_list = []
for i in range(X_test.shape[0]):
  inputs = np.array(X_test[i].reshape(-1,1))
  prediction_list.append(nn.predict(inputs))

correct_counter = 0
for i in range(len(prediction_list)):
  out_index = np.where(prediction_list[i]==np.amax(prediction_list[i]))[0][0]

  if y_test[i][out_index] == 1:
    correct_counter+=1
accuracy = correct_counter/10000

print("Accuracy is : ",accuracy*100,"% ")

Accuracy is :  90.16 % 
